# Breadth First Search (BFS) 

As the name implies, BFS traverses the given graph in a breadth first manner. Starting at a specified vertex, the algorithms iteratively searches neighboring vertices.  



@see https://en.wikipedia.org/wiki/Breadth-first_search


To compute BFS in cuGraph use: __bfs(G, start_id)__

* __G__: A cugraph.Graph object
* __start_id__ : the starting vertex ID

Returns:

* __df__: cudf.DataFrame with three names columns:
    * df["vertex"]:   The vertex id.
    * df["distance"]: The distance to the starting vertex
    * df["predecessor"]: The vertex ID of the vertex that was used to reach this vertex



___

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

The test data is samll so that results can be visually verified

## cuGraph 0.6 Notice 
cuGraph version 0.6 has a number of limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations


In [1]:
# First step is to import the needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [2]:
# define a print path function that take the dataframe and a vertex ID

def print_path(df, id):
    
    # Use the BFS predecessors and distance to trace the path 
    # from vertex id back to the starting vertex ( vertex 1 in this example)
    dist = df['distance'][id]
    lastVert = id
    for i in range(dist):
        nextVert = df['predecessor'][lastVert]
        d = df['distance'][lastVert]
        print("Vertex: " + str(lastVert) + " was reached from vertex " + str(nextVert) + 
        " and distance to start is " + str(d) )
        lastVert = nextVert

# Read the data using cuDF

In [4]:
# Read the data file
datafile='../data/networks/karate-data.csv'

cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )


In [5]:
# Let's look at the DataFrame. There should be two columns and 154 records
gdf

<cudf.DataFrame ncols=2 nrows=154 >

In [7]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

,src,dst
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6


As you can see in the output, the starting vertex ID is 1.  For the BFS algorithm that is okay.   
cuGraph will add an isolated vertex with an ID of zero.  It will not be reachable from the test dataset  

In [8]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [9]:
# Call BFS on the graph starting from vertex 1
df = cugraph.bfs(G,1)

In [10]:
# Let's take a looks at the structure of the returned dataframe
df.dtypes

vertex         int32
distance       int32
predecessor    int32
dtype: object

In [11]:
print_path(df, 22)

Vertex: 22 was reached from vertex 1 and distance to start is 1


In [12]:
print_path(df, 30)

Vertex: 30 was reached from vertex 33 and distance to start is 3
Vertex: 33 was reached from vertex 3 and distance to start is 2
Vertex: 3 was reached from vertex 1 and distance to start is 1


### Since we can see in graph illustraion above that vertex 17 is at the edge of the graph, let's run BFS with that as the startring vertex

In [13]:
# Call BFS on the graph starting from vertex 17
df2 = cugraph.bfs(G,17)

In [23]:
# Print the max distance
df2["distance"].max()

2147483647

Notice that max returned an unecpetced value.  That is becouse the isoluated vertex, 0, is unreachable.
When ever a graph contains disjointed components, the distance to the unconnected vertices will always be max_int

In [21]:
df2["distance"][0]

2147483647

In [15]:
# Print path to vertex 30
print_path(df2, 30)

Vertex: 30 was reached from vertex 33 and distance to start is 5
Vertex: 33 was reached from vertex 3 and distance to start is 4
Vertex: 3 was reached from vertex 1 and distance to start is 3
Vertex: 1 was reached from vertex 6 and distance to start is 2
Vertex: 6 was reached from vertex 17 and distance to start is 1


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___